# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csv_path = "output.csv"
csv = "output.csv"
weather_data_df = pd.read_csv("output.csv")
weather_data_df

,city,lat,long,temp,humidity,cloud,wind,country,date and time
0,koppa,13.5304,75.3633,66.36,70,0,1.77,IN,1615145247
1,upernavik,72.7868,-56.1549,-17.39,83,0,12.57,GL,1615145247
2,dikson,73.5069,80.5464,-16.33,88,100,10.36,RU,1615145247
3,port moresby,-9.4431,147.1797,73.40,100,20,5.75,PG,1615145247
4,lichinga,-13.3128,35.2406,65.19,100,90,11.50,MZ,1615145215
...,...,...,...,...,...,...,...,...,...
580,comodoro rivadavia,-45.8667,-67.5000,73.40,43,40,36.82,AR,1615145239
581,wangjing,24.6000,94.0333,54.84,66,13,0.81,IN,1615145382
582,socorro,31.6546,-106.3033,64.36,27,20,5.75,US,1615145382
583,mandalgovi,45.7625,106.2708,28.22,79,0,5.86,MN,1615145383


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [13]:
gmaps.configure(api_key=g_key)

coordinates =weather_data_df[["lat", "long"]]
humidity = weather_data_df["humidity"].astype(float)

fig = gmaps.figure()
figure_layout = {
    'width': '400px',
    'height': '400px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto',
    'zoom' : '8x'
}

# Create heat layer
heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=3)
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
vacation_df = weather_data_df
# Create the names and bins for temperature
temp_bins = [-100,70, 80, 400]
temp_group_names = ["too cool", "70-80", "too hot"]

# Create the names and bins for wind
wind_bins = [-5, 10, 500]
wind_group_names = ["less than 10 mph", "more than 10 mph"]

# Create the names and bins for cloudiness
cloud_bins = [-5, 0.1, 500]
cloud_group_names = ["clear", "cloudy"]


vacation_df["temp"] = pd.cut(vacation_df["temp"], temp_bins, labels=temp_group_names, include_lowest=True)
vacation_df["wind"] = pd.cut(vacation_df["wind"], wind_bins, labels=wind_group_names, include_lowest=True)
vacation_df["cloud"] = pd.cut(vacation_df["cloud"], cloud_bins, labels=cloud_group_names, include_lowest=True)
vacation_df

,city,lat,long,temp,humidity,cloud,wind,country,date and time
0,koppa,13.5304,75.3633,too cool,70,clear,less than 10 mph,IN,1615145247
1,upernavik,72.7868,-56.1549,too cool,83,clear,more than 10 mph,GL,1615145247
2,dikson,73.5069,80.5464,too cool,88,cloudy,more than 10 mph,RU,1615145247
3,port moresby,-9.4431,147.1797,70-80,100,cloudy,less than 10 mph,PG,1615145247
4,lichinga,-13.3128,35.2406,too cool,100,cloudy,more than 10 mph,MZ,1615145215
...,...,...,...,...,...,...,...,...,...
580,comodoro rivadavia,-45.8667,-67.5000,70-80,43,cloudy,more than 10 mph,AR,1615145239
581,wangjing,24.6000,94.0333,too cool,66,cloudy,less than 10 mph,IN,1615145382
582,socorro,31.6546,-106.3033,too cool,27,cloudy,less than 10 mph,US,1615145382
583,mandalgovi,45.7625,106.2708,too cool,79,clear,less than 10 mph,MN,1615145383


In [5]:
ideal_vacation_df=vacation_df.loc[vacation_df["temp"]=="70-80"]
ideal_vacation_df=ideal_vacation_df.loc[ideal_vacation_df["wind"]=="less than 10 mph"]
ideal_vacation_df=ideal_vacation_df.loc[ideal_vacation_df["cloud"]=="clear"]
hotel_df =ideal_vacation_df.loc[ideal_vacation_df["country"]=="IN"].copy()
hotel_df

,city,lat,long,temp,humidity,cloud,wind,country,date and time
225,kaman,27.6500,77.2667,70-80,23,clear,less than 10 mph,IN,1615145299
256,aurangabad,19.8833,75.3333,70-80,38,clear,less than 10 mph,IN,1615145308
328,port blair,11.6667,92.7500,70-80,79,clear,less than 10 mph,IN,1615145326
330,kharhial,20.2833,82.7667,70-80,18,clear,less than 10 mph,IN,1615145326
449,shahpura,27.3833,75.9667,70-80,25,clear,less than 10 mph,IN,1615145354
481,kesinga,20.2000,83.2333,70-80,19,clear,less than 10 mph,IN,1615145362
510,dod ballapur,13.2919,77.5431,70-80,40,clear,less than 10 mph,IN,1615145368
570,kagal,16.5833,74.3167,70-80,39,clear,less than 10 mph,IN,1615145380


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel = []
locations = hotel_df["city"]
# geocoordinates

target_radius = 5000
target_type = "lodging"

for index,row in hotel_df.iterrows():
    target_coordinates = f'{row["lat"]},{row["long"]}'
# set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

# base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary

    
    response = requests.get(base_url, params=params)
#hotel.append(response['hotel'])
    #print(response.json())
    hotel_response=response.json()
    hotel.append(hotel_response["results"][0]["name"])
hotel_df["Hotel Name"]=""
hotel_df.loc[:,"Hotel Name"]=hotel

hotel_df

,city,lat,long,temp,humidity,cloud,wind,country,date and time,Hotel Name
225,kaman,27.6500,77.2667,70-80,23,clear,less than 10 mph,IN,1615145299,Dr.kishan kaman
256,aurangabad,19.8833,75.3333,70-80,38,clear,less than 10 mph,IN,1615145308,OYO Rooms 012 Kushal Nagar
328,port blair,11.6667,92.7500,70-80,79,clear,less than 10 mph,IN,1615145326,Welcomhotel Bay Island Port Blair
330,kharhial,20.2833,82.7667,70-80,18,clear,less than 10 mph,IN,1615145326,Jeevan Chhaya
449,shahpura,27.3833,75.9667,70-80,25,clear,less than 10 mph,IN,1615145354,Shahpura Haveli
481,kesinga,20.2000,83.2333,70-80,19,clear,less than 10 mph,IN,1615145362,Hotel Jayashree
510,dod ballapur,13.2919,77.5431,70-80,40,clear,less than 10 mph,IN,1615145368,OYO 40115 J P Palace
570,kagal,16.5833,74.3167,70-80,39,clear,less than 10 mph,IN,1615145380,Hotel Raj Deluxe


In [7]:
hotel_df=hotel_df.rename(columns={"city":"City", "country":"Country", "lat":"Lat", "long":"Lng"})


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations,
    info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)
fig
                             
# Display figure


Figure(layout=FigureLayout(height='420px'))